In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

import pickle

In [3]:
## HACK get rid of it
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

In [10]:
list(Path(r"/data/ScannerStudy/UKSH-HE/").glob("*.tif"))

[PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-3 F1 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-14 H1 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-13 X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-7 G X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-11 H1 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-7 F1 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-11 D1 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-14 1G X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-8 E X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-14 G1 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-14 1F X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-A1-5 40X.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-15 X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-3 D X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-AI-4 D X40 IHC.tif'),
 PosixPath('/data/ScannerStudy/UKSH-HE/CRC-

In [11]:
paths = list(Path(r"D:/Datasets/ColonCancer/UKSH-HE/").glob("*.tif"))

if len(paths) == 0:
    paths = list(Path(r"/data/ScannerStudy/UKSH-HE/").glob("*.tif"))

source_slide_paths = [path for path in paths if "IHC" in path.stem]
target_slide_paths = [path for path in paths if "IHC" not in path.stem]

In [26]:
source_slide_dict = {name.stem.split("X40")[0]
                     .replace("AI", "A1").replace("1D", "D1")
                     .replace("1F", "F1").replace("1E", "E1")
                     .replace("1G", "G1").replace("1H", "H1") 
                     .replace(" H ", " H1 ").replace(" E ", " E1 ") 
                     .replace(" D ", " D1 ").replace(" F ", " F1 ").replace(" G ", " G1 ") 
                     .replace("CRC-A1-1 D1 ", "CRC-A1-1 ")
                     .replace("CRC-A1-4 D1 ", "CRC-A1-4 ")
                     .replace("CRC-A1-5 D1 ", "CRC-A1-5 ")
                     : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths if "CRC-A1-7 C " not in name.stem}
source_slide_dict.keys()

dict_keys(['CRC-A1-13 ', 'CRC-A1-7 G1 ', 'CRC-A1-14 G1 ', 'CRC-A1-8 E1 ', 'CRC-A1-14 F1 ', 'CRC-A1-15 ', 'CRC-A1-3 D1 ', 'CRC-A1-4 ', 'CRC-A1-11 D1 ', 'CRC-A1-8 H1 ', 'CRC-A1-11 H1 ', 'CRC-A1-7 C ', 'CRC-A1-7 E1 ', 'CRC-A1-7 D1 ', 'CRC-A1-14 H1 ', 'CRC-A1-5 ', 'CRC-A1-11 F1 ', 'CRC-A1-10 ', 'CRC-A1-3 F1 ', 'CRC-A1-7 F1 ', 'CRC-A1-14 E1 ', 'CRC-A1-1 ', 'CRC-A1-2 '])

In [27]:
target_slide_dict = {name.stem.split("40X")[0]
                     .replace("AI", "A1").replace("1D", "D1")
                     .replace("1F", "F1").replace("1E", "E1")
                     .replace("1G", "G1").replace("1H", "H1") : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_dict.keys()

dict_keys(['CRC-A1-3 F1 ', 'CRC-A1-14 H1 ', 'CRC-A1-11 H1 ', 'CRC-A1-7 F1 ', 'CRC-A1-11 D1 ', 'CRC-A1-14 G1 ', 'CRC-A1-5 ', 'CRC-A1-8 H1 ', 'CRC-A1-7 D1 ', 'CRC-A1-11 F1 ', 'CRC-A1-8 E1 ', 'CRC-A1-1 ', 'CRC-A1-10 ', 'CRC-A1-4 ', 'CRC-A1-3 D1 ', 'CRC-A1-7 G1 ', 'CRC-A1-14 E1 ', 'CRC-A1-15 ', 'CRC-A1-2 ', 'CRC-A1-13 ', 'CRC-A1-7 E1 ', 'CRC-A1-14 F1 '])

In [28]:
for name in source_slide_dict:
    if name not in target_slide_dict:
        print(name)

CRC-A1-7 C 


In [15]:
for name in target_slide_dict:
    if name not in target_slide_dict:
        print(name)

In [34]:
%%time

results = {}
#target_depth = 2

for thumbnail_size in [(2048, 2048)]: #(1024, 1024), (2048, 2048), (4096, 4096)
    
    for target_depth in [2]: # , 1, 2
        
        result_folder = Path(f"Depth_{target_depth}/{thumbnail_size[0]}")

        result_folder.mkdir(parents=True, exist_ok=True)
    
        for name, source in source_slide_dict.items():
            
            if name == "CRC-A1-7 C ":
                continue

            parameter = {
                # feature extractor parameters
                "point_extractor": "sift",  #orb , sift
                "maxFeatures": 1024, 
                "crossCheck": False, 
                "flann": False,
                "ratio": 0.7, 
                "use_gray": False,

                # QTree parameter 
                "homography": True,
                "filter_outliner": False,
                "debug": False,
                "target_depth": target_depth,
                "run_async": False,
                "thumbnail_size": thumbnail_size
            }

            target = target_slide_dict[name]


            source_slide = source["slide"]
            target_slide = target["slide"]

            source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
            target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])

            qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, **parameter)

            results[source["name"]] = qtree
            
            with open(result_folder / f'{qtree.source_name}-To-{qtree.target_name}.pickle', 'wb') as handle:
                pickle.dump(qtree, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/bronzi/env/EIPH_WSI/lib/python3.8/site-packages/probreg/transformation.py:73: RuntimeWarning: invalid value encountered in add
  return self.xp.dot(points, self.b.T) + self.t


CPU times: user 2h 7min 37s, sys: 3min 8s, total: 2h 10min 46s
Wall time: 43min 36s


In [39]:
"""
for idx, (path, qtree) in enumerate(results.items()):
    
    #if qtree.target_depth != 2:
    #    continue
    
    qtree.draw_feature_points(num_sub_pic=5, figsize=(12, 12))
    
    if idx == 5:
        break
"""

'\nfor idx, (path, qtree) in enumerate(results.items()):\n    \n    #if qtree.target_depth != 2:\n    #    continue\n    \n    qtree.draw_feature_points(num_sub_pic=5, figsize=(12, 12))\n    \n    if idx == 5:\n        break\n'

In [36]:
for path, qtree in results.items():
    
    if qtree.target_depth != 2:
        continue
        
    print(qtree)

Source: CRC-AI-13 X40 IHC 
Target: CRC-A1-13 40X 
Source: (0.00, 0.00, 20496.00, 9920.00) Target: (0.00, 0.00, 20032.00, 9952.00)
x: [0.995, 0.004, -866.245], y: [-0.007, 0.991, -333.311]] error: 141.083
nw: Source: (0.00, 0.00, 10248.00, 4960.00) Target: (-0.08, -0.20, 9354.08, 4508.20)
  x: [1.000, 0.007, -884.987], y: [-0.008, 0.999, -337.720]] error: 83.244
  nw: Source: (0.00, 0.00, 5124.00, 2480.00) Target: (-0.14, -0.02, 4256.14, 2096.02)
    x: [1.046, 0.136, -1105.103], y: [-0.016, 1.120, -409.057]] error: 406.808
  ne: Source: (5124.00, 0.00, 10248.00, 2480.00) Target: (4255.61, -0.02, 9396.95, 2096.02)
    x: [0.982, -0.073, 115.681], y: [0.001, 0.953, -362.908]] error: 93.830
  se: Source: (5124.00, 2480.00, 10248.00, 4960.00) Target: (4255.61, 2095.06, 9396.95, 4529.01)
    x: [1.001, 0.019, -26.593], y: [-0.001, 0.964, 45.120]] error: 28.963
  sw: Source: (0.00, 2480.00, 5124.00, 4960.00) Target: (-0.14, 2095.06, 4256.14, 4529.01)
    x: [1.001, 0.010, -872.034], y: [-0.0